In [1]:
from qiskit import *
from qiskit.quantum_info import SparsePauliOp, Statevector

import numpy as np
import networkx as nx
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info import Pauli, SparsePauliOp

from math import cos, sin, cosh, sinh, atan, exp, pi
from scipy.optimize import minimize

import sys

import copy
import pickle

# from qubo_hamiltonian import *

import itertools

/tmp/ipykernel_6326/126281046.py:7: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import PauliSumOp


In [2]:
def partition_N(N:int):
    '''do the partition of a complete graph with N vertex, to find the optimal orders for edges to run circuit in parallel
    Args:
        N: number of qubits
    Return:
        pairs_all: list of qubit index pairs (edges) in a order to parallel the circuit
    '''
    indexs = range(N)
    pairs_all = []  

    ## swap indexes of even layer [0,1,2,3,4] -> [1,0,3,2,4]
    swap_even = [i + pow(-1, i) for i in range(N - (N%2))]  
    if (N%2) == 1:
        swap_even.append(N-1)
    ## swap indexes of even layer [0,1,2,3,4] -> [0,2,1,4,3]
    swap_odd = [0]
    swap_odd.extend([i + pow(-1, i+1) for i in range(1,N-(N+1)%2)])
    if (N%2) == 0:
        swap_odd.append(N-1)
    
    ## qubit pairs need to be implemented in layer 0
    pairs_even = [(i, i+1) for i in range(0, N-1, 2)]  
    pairs_all.append(pairs_even)
    indexs = np.array(indexs)[swap_even]   ### indexs after swap even

    for i in range(1, N):
        if (i%2)==1: ## odd layer
            pair_odd = [(indexs[i], indexs[i+1]) for i in range(1, N-1, 2)]
            pairs_all.append(pair_odd)
            indexs = np.array(indexs)[swap_odd]   ### indexs after swap odd

        elif (i%2)==0: ## even layer
            pair_even = [(indexs[i], indexs[i+1]) for i in range(0, N-1, 2)]
            pairs_all.append(pair_even)
            indexs = np.array(indexs)[swap_even]   ### indexs after swap even

    return pairs_all

def find_light_cone(edges_columns):
    """find the lightcone of all qubit pairs in previous one layer"""
    lightcone_dict = {}
    for index, pair_list in enumerate(edges_columns):
        for pair in pair_list:
            qi, qj = pair
            relevent_pairs = []  ##  qubit pairs in the previous layer that in the lightcone of the current pair
            if index > 0:
                for pair_layerm1 in edges_columns[index-1]: ## qubit pairs in the previous layer
                    if (qi in pair_layerm1) or (qj in pair_layerm1):
                        relevent_pairs.append(pair_layerm1)
            lightcone_dict[pair] = relevent_pairs
    return lightcone_dict

def find_light_cone2(edges_columns, pair_now, lnow, L):
    """find the pairs in lightcone including previous L layers for a specific qubit pair at layer lnow """
    lightcone = []
    qubit_list = list(pair_now) ## qubit index included, which is used to find the pair in lightcone in previous layer
    if L == 0:
        lightcone.append(pair_now)
    else:
        for index in range(lnow-1, max(0, lnow-L)-1, -1):
            pair_list = edges_columns[index]
            for pair in pair_list:
                if pair[0] in qubit_list or pair[1] in qubit_list:
                    lightcone.append(pair)
                    qubit_list.extend(pair)
        lightcone.append(pair_now)
    return lightcone


In [3]:
def ITE(N:int, edge_coeff_dict:dict, tau:float, eigen_list:np.array):
    
    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
        
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    ## Z term 
    for i in range(N):
        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)
            
    ## ZZ term 
    state = Statevector(circ)
    for edge, coeff in edge_coeff_dict.items():
        if len(edge) == 2:
            tauc = tau * coeff
            state = circuit_update_zz(edge, state, tauc, N)

    state = np.array(state)
    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss
    return exp_poss_dict, state

In [5]:
def circuit_update_zz(Edge,State, Tauc, N):
    '''update the state vector by applying exp(-tauc * ZZ) term to the current state vector'''
    i = min(Edge)
    j = max(Edge)

    zzop = SparsePauliOp.from_sparse_list([('ZZ', [j, i], -sinh(Tauc) )], N)
    zzop += SparsePauliOp.from_list([("I"*N, cosh(Tauc))])
    
    State = State.evolve(zzop)

    # Normalize the state vector manually
    norm = np.linalg.norm(State.data)
    State = State / norm

    # Print the normalized state vector
    # print("Normalized state:", State)

    return State

In [4]:
def circuit_update_theta(Edge, State, paras, N):
    
    j = min(Edge)    #I SWAPPED i AND j TO MATCH WITH YAHUI CIRCUIT
    i = max(Edge)

    zyop = SparsePauliOp.from_sparse_list([('ZY', [j, i], -1j*sin(paras[0]/2))], N)
    zyop += SparsePauliOp.from_list([("I"*N, cos(paras[0]/2))])

    yzop = SparsePauliOp.from_sparse_list([('YZ', [j, i], -1j*sin(paras[1]/2))], N)
    yzop += SparsePauliOp.from_list([("I"*N, cos(paras[1]/2))])

    op = zyop.compose(yzop)

    if len(paras) == 4:
        iyop = SparsePauliOp.from_sparse_list([('IY', [j, i], -1j*sin(paras[2]/2))], N)
        iyop += SparsePauliOp.from_list([("I"*N, cos(paras[2]/2))])
        op = op.compose(iyop)

        yiop = SparsePauliOp.from_sparse_list([('YI', [j, i], -1j*sin(paras[3]/2))], N)
        yiop += SparsePauliOp.from_list([("I"*N, cos(paras[3]/2))])
        op = op.compose(yiop)
    elif len(paras) == 6:
        xyop = SparsePauliOp.from_sparse_list([('XY', [j, i], -1j*sin(paras[4]/2))], N)
        xyop += SparsePauliOp.from_list([("I"*N, cos(paras[4]/2))])
        op = op.compose(xyop)

        yxop = SparsePauliOp.from_sparse_list([('YX', [j, i], -1j*sin(paras[5]/2))], N)
        yxop += SparsePauliOp.from_list([("I"*N, cos(paras[5]/2))])
        op = op.compose(yxop)
    
    State = State.evolve(op)

    # print('states2', State)

    return State

In [ ]:
def square_modulus_cost_light_cone(Paras : list, *args):

    # para_init = np.zeros((len(lightcone_dict[edge]) + 1, 2))

    Edge_list = args[0]
    State = args[1]
    Tauc = args[2]
    Updated_state = args[3]
    N = args[4]

    State_zz = circuit_update_zz(Edge_list[-1], Updated_state, Tauc, N)
    
    if len(Paras) == 2*len(Edge_list):
        for index, edge in enumerate(Edge_list):
            Parameters = [Paras[2*index], Paras[2*index + 1]]
            State = circuit_update_theta(edge, State, Parameters, N)
    elif len(Paras) == 4*len(Edge_list):
        for index, edge in enumerate(Edge_list):
            Parameters = [Paras[4*index], Paras[4*index + 1], Paras[4*index + 2], Paras[4*index + 3]]
            State = circuit_update_theta(edge, State, Parameters, N)
    elif len(Paras) == 6*len(Edge_list):
        for index, edge in enumerate(Edge_list):
            Parameters = [Paras[6*index], Paras[6*index + 1], Paras[6*index + 2], Paras[6*index + 3], Paras[6*index + 4], Paras[6*index + 5]]
            State = circuit_update_theta(edge, State, Parameters, N)

    # Compute the scalar product (inner product) between the two state vectors
    inner_product = State_zz.inner(State)

    # Compute the square modulus of the inner product
    square_modulus = abs(inner_product)**2

    return - square_modulus

In [ ]:
def warm_start_parameters_lightcone(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list, lightcone_dict: dict):    

    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    # tau = 0.2

    # Z term
    for i in range(N):

        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)
        
    ## ZZ term 
    state = Statevector(circ)
    updated_state = state 

    #print('\nnumber of columns is:', len(edges_columns))

    for column_index, column in enumerate(edges_columns):
        # print('\n##################################################')
        # print('column index', column_index, 'column', column)

        if column_index == 0: 

            first_column_state = state

            for edge in column:
                
                if len(lightcone_dict[edge]) != 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()
            
                # print('\nedge', edge)  

                tauc = tau * edge_coeff_dict[edge]

                para_init = [0,0]

                final = minimize(square_modulus_cost,
                                    para_init,
                                    args = (edge, first_column_state, tauc, N),
                                    jac=False,
                                    bounds=None,
                                    method='L-BFGS-B',
                                    callback=None,
                                    options={'maxiter': 1000})

                para = final.x

                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                edge_params_dict[edge] = para
                # print('edge_params_dict:', edge_params_dict)

                # print('old state', state)
                first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT

                # print('first column state - new state', first_column_state)
            
        else:
            
            for edge in column: 
                if len(lightcone_dict[edge]) == 0:
                        sys.stderr.write('something is wrong with the lightcones')
                        sys.exit()

                # print('\nedge', edge)  

                # print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

                # print('initial state', state)

                updated_state = copy.deepcopy(state)
                
                for old_edge in lightcone_dict[edge]:
                    # print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
                
                # print('updated_state', updated_state)

                edge_list = lightcone_dict[edge] + [edge]
                # print('edge list', edge_list)
                
                tauc = tau * edge_coeff_dict[edge]

                para_init = np.zeros(2 + 2*len(lightcone_dict[edge]))
                # print('para init', para_init)

                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = (edge_list, state, tauc, updated_state, N),
                        jac=False,
                        bounds=None,
                        method='L-BFGS-B',
                        callback=None,
                        options={'maxiter': 1000})

                para = final.x
                # print('opt paramenters', para)
                # print('final square modulus', final.fun)

                para = (np.array(para)).reshape(-1, 2)
                # print('param list reshaped', para)
                for index, edge in enumerate(edge_list):
                    edge_params_dict[edge] = para[index] 
                # print('edge_params_dict', edge_params_dict)
            
            # print('\n######### previous column update ########')
            # print('column index', column_index, 'column', column)
            # print('previous column is:', edges_columns[column_index -1])

            for edge in edges_columns[column_index -1]:
                # print('edge', edge, 'parameter', edge_params_dict[edge])
                state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

            # print('circuit updated ad the previous column i.e. column', column_index -1)
            # print('updated statevector', state)

    # print('\n######### last column update ########')
    # print('last column is:', edges_columns[ -1])

    for edge in edges_columns[ -1]:
        # print('edge', edge, 'parameter', edge_params_dict[edge])
        state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    # print('circuit updated at the last column i.e.', len(edges_columns) - 1)
    # print('updated statevector', state)

    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    # Concatenate and flatten all arrays into a single array
    flattened_array = np.concatenate(values_as_arrays)
    # Convert the flattened array to a list if needed
    params_list = flattened_array.tolist()
    # print(' params_list' , params_list )

    state = np.array(state)

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss

    return edge_params_dict, params_list, exp_poss_dict, state

In [ ]:
def warm_start_parameters_old(N : int, tau:float, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list):
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])


    # Z term
    for i in range(N):

        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)

        edge_params_dict[(i,)] = para
        circ.ry(para, i)
        
    ## ZZ term 

    state = Statevector(circ)
    first_column_state = state

    for column_index, column in enumerate(edges_columns):
        for edge in column:
            tauc = tau * edge_coeff_dict[edge]
            para_init = [0.00,0.00]
            final = minimize(square_modulus_cost,
                                para_init,
                                args = (edge, first_column_state, tauc, N),
                                jac=False,
                                bounds=None,
                                method='L-BFGS-B',
                                callback=None,
                                options={'maxiter': 100})

            para = final.x

            # print('opt paramenters', para)
            # print('final square modulus', final.fun)

            edge_params_dict[edge] = para
            
            #print('old state', state)
            first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT


    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    flattened_array = np.concatenate(values_as_arrays)
    params_list = flattened_array.tolist()

    state = np.array(first_column_state)
    eigens_ids = np.argsort(eigen_list)[:100]

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss
   
    return edge_params_dict, params_list, exp_poss_dict, state

In [ ]:
def warm_start_parameters_adaptlightcone(N : int, tau:float, numpara:int, ampth:float, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list, lightcone_dict: dict):
    '''' warm start the parameters for the circuit using the lightcone adaptivly
    Args:
        N: number of qubits
        tau: imaginary time
        numpara: number of parameters for each edge, 2, 4, or 6
        ampth: amplitude threshold, if the amplitude between optimized state and ITE state is larger than this value, the optimization stops; \
               otherwise continue with a larger lightcone including more previous layers
        edge_coeff_dict: coefficients for each edge
        edges_columns: list of edges in each layer
        eigen_list: list of eigenvalues
    Return:
        edge_params_dict: parameters for each edge
        params_list: list of parameters
        exp_poss_dict: probabilities of eigenvalues using warm start circuit
        state: final state vector 
    '''
    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit
    edge_coeff_dict_ite = {} ## record the edges has been executed, to get the ite state for each step

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    ## Z term 
    for i in range(N):
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)

        edge_coeff_dict_ite[(i,)] = edge_coeff_dict[(i,)]
   
    
    ## ZZ term 
    state = Statevector(circ)
    for column_index, column in enumerate(edges_columns):
        print('\n### column index', column_index, 'column', column)
        if column_index == 0: 
            first_column_state = copy.deepcopy(state)
            for edge in column:
                if len(lightcone_dict[edge]) != 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()
                edge_coeff_dict_ite[edge] = edge_coeff_dict[edge]
                _, state_ite = ITE(N, edge_coeff_dict_ite, tau, eigen_list)

                tauc = tau * edge_coeff_dict[edge]

                para_init = [0]* numpara
                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = ([edge], first_column_state, tauc, first_column_state, N),
                        jac=False,
                        bounds=None,
                        method='SLSQP',
                        callback=None,
                        options={'maxiter': 1000})
                para = final.x
                print('\nedge: ', edge,  ',  coeff: ', edge_coeff_dict[edge],  ',     final square modulus: ', final.fun)

                edge_params_dict[edge] = para

                #region check the overlap between the ite state and the optimized state
                state_opt = copy.deepcopy(state)
                for old_edge in edge_coeff_dict_ite.keys():
                    if len(old_edge) == 2:
                        state_opt = circuit_update_theta(old_edge, state_opt, edge_params_dict[old_edge], N)
                state_opt = np.array(state_opt)
                print('overlap between ite and optimized state: ', state_ite.T.conj() @ state_opt)
                
        else:
            for ei, edge in enumerate(column):  
                print('\nedge:', edge, ',  coeff:', edge_coeff_dict[edge])
                edge_coeff_dict_ite[edge] = edge_coeff_dict[edge]
                _, state_ite = ITE(N, edge_coeff_dict_ite, tau, eigen_list)
                ## optimize the gate in the lightcone including previous L layers
                for L in range(0, column_index+1):
                    lightcone = find_light_cone2(edges_columns, edge, column_index, L)  ## current edge is included in the lightcone, which is the last element in the list
                    print('~~L:', L, 'lightcone:', lightcone)

                    updated_state = copy.deepcopy(state)
                    optimized_state = copy.deepcopy(state)

                    para_init = []
                    edge_list = []  ## to save the edges in the lightcone that will be optimized
                    for ci in range(column_index-L):## previours layer before lightcone
                        for old_edge in edges_columns[ci]:
                            updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)  
                            optimized_state = circuit_update_theta(old_edge, optimized_state, edge_params_dict[old_edge], N)
                    for ci in range(column_index-L, column_index):## layer in lightcone      
                        for old_edge in edges_columns[ci]:
                            if old_edge in lightcone:
                                edge_list.append(old_edge)
                                para_init.extend(list(edge_params_dict[old_edge]))
                                updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N) 
                    
                    tauc = tau * edge_coeff_dict[edge]

                    edge_list = edge_list + [edge]
                    para_init = para_init + [0]*numpara
                    #print('optimize edge list', edge_list, ',  para_init: ', para_init)
                    final = minimize(square_modulus_cost_light_cone,
                            para_init,
                            args = (edge_list, optimized_state, tauc, updated_state, N),
                            jac=False,
                            bounds=None,
                            method='SLSQP',
                            callback=None,
                            options={'maxiter': 1000})

                    para = final.x
                    print('final square modulus: ', final.fun)

                    ## check the overlap between the ite state and the optimized state
                    para = (np.array(para)).reshape(-1, numpara)
                    edge_para_dict = {edge:para[index] for index, edge in enumerate(edge_list)}
                    for ci in range(column_index-L, column_index+1):## layer in lightcone      
                        for old_edge in edges_columns[ci]:
                            if old_edge in edge_coeff_dict_ite.keys():
                                if old_edge in lightcone:
                                    optimized_state = circuit_update_theta(old_edge, optimized_state, edge_para_dict[old_edge], N)
                                else:
                                    optimized_state = circuit_update_theta(old_edge, optimized_state, edge_params_dict[old_edge], N)
                    optimized_state = np.array(optimized_state)
                    print('overlap between ite and optimized state: ', state_ite.T.conj() @ optimized_state)

                    if final.fun < -ampth:  ##
                        break

                ## update the parameters for the optimized edge
                for index, edge in enumerate(edge_list):
                    edge_params_dict[edge] = para[index]
                ## check the overlap between the ite state and the optimized state
                state_opt = copy.deepcopy(state)
                for old_edge in edge_coeff_dict_ite.keys():
                    if len(old_edge) == 2:
                        state_opt = circuit_update_theta(old_edge, state_opt, edge_params_dict[old_edge], N)
                state_opt = np.array(state_opt)
                print('overlap between ite and final optimized state: ', state_ite.T.conj() @ state_opt)
                 
                

    ## get the final state vector
    for ci in range(len(edges_columns)):
        for edge in edges_columns[ ci]:
            state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    # Concatenate and flatten all arrays into a single array
    flattened_array = np.concatenate(values_as_arrays)
    # Convert the flattened array to a list if needed
    params_list = flattened_array.tolist()
    # print(' params_list' , params_list )

    state = np.array(state)

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss
    return edge_params_dict, params_list, exp_poss_dict, state

In [ ]:
def Hamiltonian_qubo(N, edge_list, h_list, J_list):
    """Hamiltonian defined by a N vertex graph with connected edge in edge_list
    Args:
        N: number of qubits
        edge_list: list of edges(qubit index pairs)
        h_list: coefficients of single Pauli Z term
        J_list: coefficients of ZZ term
    Return:
        H: PauliSumOp, Hamiltonian

    """
    pauli_list = []
    for i in range(N):
        pauli_str = (N-i-1)*'I' + 'Z' + i*'I'
        op = Pauli(pauli_str)
        pauli_list.append((op.to_label(), h_list[i]))
        
    for k, (i, j) in enumerate(edge_list):
        x_p = np.zeros(N, dtype = bool)
        z_p = np.zeros(N, dtype = bool)
        z_p[i] = True
        z_p[j] = True
        op = Pauli((z_p, x_p))
        pauli_list.append((op.to_label(), J_list[k]))
        
    H = PauliSumOp.from_list(pauli_list)
    
    return H

# Generate instances

In [ ]:
N = 6
tau = 1
for r in range(10):
    print('\n\n#############################################################################')
    print('r:', r)
    #region load qubo instances, get Hamiltonian and edge_coeff_dict
    instance_dir = './instances/complete/N_' + str(N )
    with open(instance_dir + '/QUBO_' + str(N ) + 'V_comp_'+ '.gpickle', 'rb') as f:
        G = pickle.load(f)

    coeff_list = np.loadtxt(instance_dir + '/QUBO_coeff_' + str(N) + 'V_comp_'+ 'r_'+ str(r)+ '.txt')
    h_list = coeff_list[: N ]
    J_list = coeff_list[N :]

    edges_columns = partition_N(N)
    lightcone_dict = find_light_cone(edges_columns)
    edge_list = G.edges()
    pairs_all = list(itertools.chain.from_iterable(partition_N(N)))

    #extrcact Hamiltonian, edge coefficients and eigen list
    H = Hamiltonian_qubo(N, edge_list, h_list, J_list)
    eigen_list = H.to_spmatrix().diagonal().real

    # Initialize dictionary for single Pauli Z term with coefficient from h_list
    edge_coeff_dict = {}
    edge_coeff_dict.update({(i,): h_val for i, h_val in enumerate(h_list)})
    #Initialize dictionary for Pauli ZZ term with coefficient from from J_list
    edge_coeff_dict.update({edge: J_val for edge, J_val in zip(edge_list, J_list)})
    print('edge_coeff_dict' , edge_coeff_dict)
    #endregion


    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ITE')
    exp_poss_dict, state_ite = ITE(N, edge_coeff_dict, tau, eigen_list)
    print('exp_poss_dict', exp_poss_dict)

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Warm start old')
    edge_params_dict, params_init, exp_poss_dict, state_all = warm_start_parameters_adaptlightcone(N, tau, 2, 0.0, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    print('edge_params_dict', edge_params_dict)
    print('exp_poss_dict', exp_poss_dict)
    print('overlap', np.dot(state_ite, state_all))
    # edge_params_dict, params_init, exp_poss_dict, state_old = warm_start_parameters_old(N, tau, edge_coeff_dict, edges_columns, eigen_list)
    # print('exp_poss_dict', exp_poss_dict)
    # print('overlap', np.dot(state_ite, state_old))
    #region other test
    # print('\n################################ Warm start')
    # edge_params_dict, params_init, exp_poss_dict, state_lightcone = warm_start_parameters_lightcone(N, tau, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    # # print(edge_params_dict)
    # print(exp_poss_dict)
    # print('overlap', np.dot(state_ite, state_lightcone))

    # print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  Warm start YZ')
    # edge_params_dict, params_init, exp_poss_dict, state_lightcone2 = warm_start_parameters_lightcone2(N, tau, 2, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    # print('edge_params_dict', edge_params_dict)
    # print('exp_poss_dict', exp_poss_dict)
    # print('overlap', np.dot(state_ite, state_lightcone2))

    # print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  Warm start YZ_Y')
    # edge_params_dict, params_init, exp_poss_dict, state_lightcone2 = warm_start_parameters_lightcone2(N, tau, 4, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    # print('edge_params_dict', edge_params_dict)
    # print('exp_poss_dict', exp_poss_dict)
    # print('overlap', np.dot(state_ite, state_lightcone2))

    # print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  Warm start YZ_Y_XY')
    # edge_params_dict, params_init, exp_poss_dict, state_lightcone2 = warm_start_parameters_lightcone2(N, tau, 6, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    # print('edge_params_dict', edge_params_dict)
    # print('exp_poss_dict', exp_poss_dict)
    # print('overlap', np.dot(state_ite, state_lightcone2))
    #endregion

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  Warm start all YZ_Y')
    edge_params_dict, params_init, exp_poss_dict, state_all = warm_start_parameters_adaptlightcone(N, tau, 4, 0.95, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    print('edge_params_dict', edge_params_dict)
    print('exp_poss_dict', exp_poss_dict)
    print('overlap', np.dot(state_ite, state_all))

    print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  Warm start all YZ_Y_XY')
    edge_params_dict, params_init, exp_poss_dict, state_all = warm_start_parameters_adaptlightcone(N, tau, 6, 0.95, edge_coeff_dict, edges_columns, eigen_list, lightcone_dict)
    print('edge_params_dict', edge_params_dict)
    print('exp_poss_dict', exp_poss_dict)
    print('overlap', np.dot(state_ite, state_all))


In [ ]:
def warm_start_parameters_lightcone2(N : int, tau:float, numpara:int, edge_coeff_dict : dict, edges_columns :list,  eigen_list:list, lightcone_dict: dict):    

    eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
    edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
    exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

    q = QuantumRegister(N, name = 'q')
    circ = QuantumCircuit(q)
    circ.clear()
    circ.h(q[::])

    # Z term
    for i in range(N):

        #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
        tauc = tau * edge_coeff_dict[(i,)] 
        para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
        edge_params_dict[(i,)] = para
        circ.ry(para, i)
        
    ## ZZ term 
    state = Statevector(circ)
    updated_state = state 

    #print('\nnumber of columns is:', len(edges_columns))

    for column_index, column in enumerate(edges_columns):
        # print('\n##################################################')
        print('### column index', column_index, 'column', column)

        if column_index == 0: 
            first_column_state = state
            for edge in column:
                
                if len(lightcone_dict[edge]) != 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    sys.exit()

                tauc = tau * edge_coeff_dict[edge]
                para_init = [0]* numpara
                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = ([edge], first_column_state, tauc, first_column_state, N),
                        jac=False,
                        bounds=None,
                        method='SLSQP',
                        callback=None,
                        options={'maxiter': 1000})
                para = final.x
                print('edge: ', edge,  ',     final square modulus: ', final.fun)

                ## test old
                final2 = minimize(square_modulus_cost_light_cone,
                        [0, 0],
                        args = ([edge], first_column_state, tauc, first_column_state, N),
                        jac=False,
                        bounds=None,
                        method='SLSQP',
                        callback=None,
                        options={'maxiter': 1000})

                print('edge: ', edge,  ',  old final square modulus: ', final2.fun)

                edge_params_dict[edge] = para
            
        else:
            print('### column index', column_index, 'column', column)
            for edge in column: 
                if len(lightcone_dict[edge]) == 0:
                        sys.stderr.write('something is wrong with the lightcones')
                        sys.exit()


                updated_state = copy.deepcopy(state) 
                for old_edge in lightcone_dict[edge]:
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
                
                # print('updated_state', updated_state)

                edge_list = lightcone_dict[edge] + [edge]
                # print('edge list', edge_list)
                
                tauc = tau * edge_coeff_dict[edge]

                para_init = [0] * ( numpara + numpara*len(lightcone_dict[edge]))
                # print('para init', para_init)

                final = minimize(square_modulus_cost_light_cone,
                        para_init,
                        args = (edge_list, state, tauc, updated_state, N),
                        jac=False,
                        bounds=None,
                        method='SLSQP',
                        callback=None,
                        options={'maxiter': 1000})

                para = final.x
                print('\nedge: ', edge,  ',     final square modulus: ', final.fun)

                ## test old
                final2 = minimize(square_modulus_cost_light_cone,
                        [0,0],
                        args = ([edge], updated_state, tauc, updated_state, N),
                        jac=False,
                        bounds=None,
                        method='SLSQP',
                        callback=None,
                        options={'maxiter': 1000})

                print('edge: ', edge,  ',  old final square modulus: ', final2.fun)

                if abs(final.fun) > 0.0:
                    para = (np.array(para)).reshape(-1, numpara)
                    for index, edge in enumerate(edge_list):
                        edge_params_dict[edge] = para[index]
                else:
                    edge_params_dict[edge] = final2.x 
                # edge_params_dict[edge] = final2.x
            

            for edge in edges_columns[column_index -1]:
                state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    for edge in edges_columns[ -1]:
        state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

    #generate params_list
    values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
    # Concatenate and flatten all arrays into a single array
    flattened_array = np.concatenate(values_as_arrays)
    # Convert the flattened array to a list if needed
    params_list = flattened_array.tolist()
    # print(' params_list' , params_list )

    state = np.array(state)

    for id in eigens_ids:
        eigen = eigen_list[id]
        poss = abs(state[id])**2
        # print('eigen', eigen, 'poss', poss)
        exp_poss_dict[eigen] = poss

    return edge_params_dict, params_list, exp_poss_dict, state

# Test old

# Test new

## simplified

In [ ]:
tau = 1
eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

q = QuantumRegister(N, name = 'q')
circ = QuantumCircuit(q)
circ.clear()
circ.h(q[::])

## Z term 
for i in range(N):

    #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
    tauc = tau * edge_coeff_dict[(i,)] 
    para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
    edge_params_dict[(i,)] = para
    circ.ry(para, i)
        
## ZZ term 
state = Statevector(circ)
updated_state = state 

print('\nnumber of columns is:', len(edges_columns))

for column_index, column in enumerate(edges_columns[:2]):
    print('\n##################################################')
    print('column index', column_index, 'column', column)

    if column_index == 0: 
        first_column_state = state
        for edge in column:
            print('edge', edge)   
            if len(lightcone_dict[edge]) != 0:
                sys.stderr.write('something is wrong with the lightcones')
                sys.exit()
        
            tauc = tau * edge_coeff_dict[edge]
            para_init = [0,0]

            final = minimize(square_modulus_cost,
                                para_init,
                                args = (edge, first_column_state, tauc, N),
                                jac=False,
                                bounds=None,
                                method='L-BFGS-B',
                                callback=None,
                                options={'maxiter': 1000})

            para = final.x
            edge_params_dict[edge] = para
        
            #first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT
            
    else:
        for ei, edge in enumerate(column): 
            print('\nedge', edge) 
            if len(lightcone_dict[edge]) == 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    print('error edge', edge)
                    sys.exit()

            print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

            updated_state = copy.deepcopy(state)
            for old_edge in edges_columns[column_index -1]:
                print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
               

            edge_list = [edge]
            if ei > 0:
                old_edge = column[ei - 1]
                print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N) 
                # edge_list = [old_edge] + edge_list
            print('edge list', edge_list)
            
            tauc = tau * edge_coeff_dict[edge]

            #para_init = para_init + [0,0]
            para_init = [0,0] * len(edge_list)
            print('para init', para_init)

            final = minimize(square_modulus_cost_light_cone,
                    para_init,
                    args = (edge_list, updated_state, tauc, updated_state, N),
                    jac=False,
                    bounds=None,
                    method='L-BFGS-B',
                    callback=None,
                    options={'maxiter': 1000})

            para = final.x
            print('final.x', final.x)
            print('final.fun: ', final.fun)

            para = (np.array(para)).reshape(-1, 2)
            for index, edge in enumerate(edge_list):
                edge_params_dict[edge] = para[index] 
            print('edge_params_dict', edge_params_dict)
   

        for edge in edges_columns[column_index -1]:
            state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

          

for edge in edges_columns[ 1]:
    print('edge', edge, 'parameter', edge_params_dict[edge])
    state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

# # print('circuit updated at the last column i.e.', len(edges_columns) - 1)
# print('updated statevector', state)

#generate params_list
values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
# Concatenate and flatten all arrays into a single array
flattened_array = np.concatenate(values_as_arrays)
# Convert the flattened array to a list if needed
params_list = flattened_array.tolist()
print(' params_list' , params_list )

state = np.array(state)

for id in eigens_ids:
    eigen = eigen_list[id]
    poss = abs(state[id])**2
    # print('eigen', eigen, 'poss', poss)
    exp_poss_dict[eigen] = poss
print('\nedge_params_dict', edge_params_dict)
print('\nfidelity: ', exp_poss_dict)

## lightcone

In [ ]:
tau = 0.5
eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

q = QuantumRegister(N, name = 'q')
circ = QuantumCircuit(q)
circ.clear()
circ.h(q[::])

## Z term 
for i in range(N):

    #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
    tauc = tau * edge_coeff_dict[(i,)] 
    para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
    edge_params_dict[(i,)] = para
    circ.ry(para, i)
        
## ZZ term 
state = Statevector(circ)
#updated_state = state 

print('\nnumber of columns is:', len(edges_columns))

for column_index, column in enumerate(edges_columns):
    print('\n##################################################')
    print('column index', column_index, 'column', column)

    if column_index == 0: 
        first_column_state = state
        for edge in column:
            print('edge', edge)   
            if len(lightcone_dict[edge]) != 0:
                sys.stderr.write('something is wrong with the lightcones')
                sys.exit()
        
            tauc = tau * edge_coeff_dict[edge]
            para_init = [0,0]

            final = minimize(square_modulus_cost,
                                para_init,
                                args = (edge, first_column_state, tauc, N),
                                jac=False,
                                bounds=None,
                                method='L-BFGS-B',
                                callback=None,
                                options={'maxiter': 1000})

            para = final.x
            edge_params_dict[edge] = para
        
            first_column_state = circuit_update_theta(edge, first_column_state, para, N)  #THIS LINE SHOULDN'T BE HERE BUT SOMEHOW THERE IS A TINY NUMERICAL ERROR IN THE THETAS IF I DON'T UPDATE THE CIRCUIT
            
    else:
        for ei, edge in enumerate(column): 
            print('edge', edge) 
            if len(lightcone_dict[edge]) == 0:
                    sys.stderr.write('something is wrong with the lightcones')
                    print('error edge', edge)
                    sys.exit()

            print('len light cone:', len(lightcone_dict[edge]), ', light cone edges:', lightcone_dict[edge])

            updated_state = copy.deepcopy(state)
            para_init = []
            edge_list = []
            for ci in range(column_index):
                for old_edge in edges_columns[ci]:
                    #print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                    edge_list.append(old_edge)
                    para_init.extend(list(edge_params_dict[old_edge]))
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N)    
            if ei > 0:
                for old_edge in column[:ei]:
                    print('old_edge', old_edge, 'param', edge_params_dict[old_edge])
                    edge_list.append(old_edge)
                    para_init.extend(list(edge_params_dict[old_edge]))
                    updated_state = circuit_update_theta(old_edge, updated_state, edge_params_dict[old_edge], N) 
                # edge_list = [old_edge] + edge_list

            edge_list = edge_list + [edge]
            print('edge list', edge_list)
            
            tauc = tau * edge_coeff_dict[edge]

            para_init = para_init + [0,0]
            print('para init', para_init)

            final = minimize(square_modulus_cost_light_cone,
                    para_init,
                    args = (edge_list, state, tauc, updated_state, N),
                    jac=False,
                    bounds=None,
                    method='COBYLA',
                    callback=None,
                    options={'maxiter': 1000})

            para = final.x
            print('final.x', final.x)
            print('final.fun: ', final.fun)

            para = (np.array(para)).reshape(-1, 2)
            for index, edge in enumerate(edge_list):
                edge_params_dict[edge] = para[index] 
            print('edge_params_dict', edge_params_dict)
   

        # for edge in edges_columns[column_index -1]:
        #     state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

          
for ci in range(len(edges_columns)):
    for edge in edges_columns[ ci]:
        print('edge', edge, 'parameter', edge_params_dict[edge])
        state = circuit_update_theta(edge, state, edge_params_dict[edge], N)

# # print('circuit updated at the last column i.e.', len(edges_columns) - 1)
# print('updated statevector', state)

# #generate params_list
# values_as_arrays = [np.atleast_1d(value) for value in edge_params_dict.values()]
# # Concatenate and flatten all arrays into a single array
# flattened_array = np.concatenate(values_as_arrays)
# # Convert the flattened array to a list if needed
# params_list = flattened_array.tolist()
# print(' params_list' , params_list )

state = np.array(state)

for id in eigens_ids:
    eigen = eigen_list[id]
    poss = abs(state[id])**2
    # print('eigen', eigen, 'poss', poss)
    exp_poss_dict[eigen] = poss
print('\nedge_params_dict', edge_params_dict)
print('\nfidelity: ', exp_poss_dict)

# ITE

In [ ]:

eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

q = QuantumRegister(N, name = 'q')
circ = QuantumCircuit(q)
circ.clear()
circ.h(q[::])

## Z term 
for i in range(N):

    #para = get_initial_para_1op_Y(N, [i], edge_coeff_dict[(i,)], tau, circ, shots, approximation)[0] #use this to extimate para from min expectation value
    tauc = tau * edge_coeff_dict[(i,)] 
    para = 2*atan( -exp(-2*tauc) ) + pi/2 #use this to use analytic formula (only valid for 1 layer)
    edge_params_dict[(i,)] = para
    circ.ry(para, i)
        
## ZZ term 
state = Statevector(circ)

for edge in G.edges():
    coeff = edge_coeff_dict[edge]
    print('edge', edge, 'coeff', coeff)
    tauc = tau * coeff
    state = circuit_update_zz(edge, state, tauc, N)

state = np.array(state)
for id in eigens_ids:
    eigen = eigen_list[id]
    poss = abs(state[id])**2
    # print('eigen', eigen, 'poss', poss)
    exp_poss_dict[eigen] = poss
print('\nedge_params_dict', edge_params_dict)
print('\nfidelity: ', exp_poss_dict)

In [ ]:
def ITE(N, qi, coeff, tau, vec):
    """
    Caculating the statevector after imiginary time evolution, 
    i.e, e^{- tau * coeff * Z_i} * vec, or e^{- tau * coeff * Z_jZ_i} * vec
    
    Args:
        N: int, number of qubits
        qi: qubit index list, [i] for single-qubit term, [i,j] for two-qubit term
        coeff: float, coeffieicnt of the terms acting on qubit i or (i, j)
        tau: float, time for imaginary time evolution
        vec:numpy.array,  the statevector of current state
    Return:
        vec_tau / norm: numpy.array, normalized statevector after imiginary time evolution
    """
    
    if len(qi) == 1:  ## single Z term
        i = qi[0]
        Z_op = Pauli('I'*(N-1-i) + 'Z' + 'I'*i)
    elif len(qi) == 2: ## two body term Z_jZ_i
        i = min(qi)
        j = max(qi)
        Z_op = Pauli('I'*(N-1-j) + 'Z' + 'I'*(j-i-1) + 'Z' + 'I'*i)
        
    Z_op_diag = Z_op.to_matrix(sparse=True).diagonal().real
    vec_z = np.multiply(Z_op_diag, vec)  ## multiplying each element
    vec_tau = np.cosh(coeff * tau) * vec - np.sinh(coeff * tau) * vec_z
    
    norm = np.linalg.norm(vec_tau)
    
    return vec_tau / norm

In [ ]:

eigens_ids = np.argsort(eigen_list)[:100]  ## return the id of the lowest 100 eigenvalues    ????
    
edge_params_dict = {} ## to save the initial parameters for each vertex or edge in l'th layer
exp_poss_dict = {}   ## save probalities of eigenvalues using warm start circuit

vec = np.ones(2**N) / np.sqrt(2**N)

## Z term 
for i in range(N):
    coeff = edge_coeff_dict[(i,)]
    vec = ITE(N, [i], coeff, tau, vec)
        

## ZZ term
for edge in G.edges():
    coeff = edge_coeff_dict[edge]
    vec = ITE(N, edge, coeff, tau, vec)

state = vec
for id in eigens_ids:
    eigen = eigen_list[id]
    poss = abs(state[id])**2
    # print('eigen', eigen, 'poss', poss)
    exp_poss_dict[eigen] = poss
print('\nedge_params_dict', edge_params_dict)
print('\nfidelity: ', exp_poss_dict)